In [1]:
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

try:
    word_tokenize("test")
except LookupError:
    nltk.download('punkt')
nltk.download('punkt_tab')

MAX_SEQ_LEN = 300 # Максимальная длина последовательности токенов (среднее количество ~1300, медиана ~1000). Будем надеятся что в первых 300 токенах достаточно информации чтобы что-то выучить, но мне не хватает вычислительной мощности сделать больше
INPUT_DIM_FOR_MODEL = 64

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {DEVICE}")
imdb_dataset = load_dataset("stanfordnlp/imdb")
df_train = pd.DataFrame(imdb_dataset["train"])
df_test = pd.DataFrame(imdb_dataset["test"]) 
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['label'])
print(f"Размер обучающей выборки до обрезания: {len(df_train)}")
print(f"Размер валидационной выборки до обрезания: {len(df_test)}")
print(f"Размер тестовой выборки до обрезания: {len(df_test)}\n---------------------------------------")

df_train, df_val, df_test = df_train.sample(n=10000, random_state=42), df_val.sample(n=5000, random_state=42), df_test.sample(n=5000, random_state=42) # обрежем выборки, иначе модель обучается слишком долго

print(f"Итоговый размер обучающей выборки: {len(df_train)}")
print(f"Итоговый размер валидационной выборки: {len(df_test)}")
print(f"Итоговый размер тестовой выборки: {len(df_test)}")

def tokenize_text_data(texts_list): # токенизация
    tokenized_texts = []
    for text in texts_list:
        tokens = word_tokenize(text.lower()) # из nltk
        tokenized_texts.append(tokens)
    return tokenized_texts

def build_vocabulary(list_of_tokenized_texts): # словари token -> id, id -> token
    all_tokens_flat = []
    for tokens in list_of_tokenized_texts:
        all_tokens_flat.extend(tokens)
    
    word_counts = Counter(all_tokens_flat)
    most_common_words = word_counts.most_common()
    
    word_to_id = {'<PAD>': 0, '<UNK>': 1}
    id_to_word = {0: '<PAD>', 1: '<UNK>'}
    
    for index, (word, _) in enumerate(most_common_words):
        current_id = index + 2
        word_to_id[word] = current_id
        id_to_word[current_id] = word
        
    return word_to_id, id_to_word

def convert_tokens_to_ids(list_of_tokenized_texts, word_to_id_map, max_seq_l): # [token] -> [ids]
    sequences_of_ids = []
    for tokens in list_of_tokenized_texts:
        ids = [word_to_id_map.get(token, word_to_id_map['<UNK>']) for token in tokens]
        if len(ids) < max_seq_l:
            ids.extend([word_to_id_map['<PAD>']] * (max_seq_l - len(ids)))
        else:
            ids = ids[:max_seq_l]
        sequences_of_ids.append(ids)
    return np.array(sequences_of_ids)

train_texts_tokenized = tokenize_text_data(df_train['text'].tolist())
val_texts_tokenized = tokenize_text_data(df_val['text'].tolist())
test_texts_tokenized = tokenize_text_data(df_test['text'].tolist())

main_vocab, main_id2token = build_vocabulary(train_texts_tokenized)

X_train_ids = convert_tokens_to_ids(train_texts_tokenized, main_vocab, MAX_SEQ_LEN)
X_val_ids = convert_tokens_to_ids(val_texts_tokenized, main_vocab, MAX_SEQ_LEN)
X_test_ids = convert_tokens_to_ids(test_texts_tokenized, main_vocab, MAX_SEQ_LEN)

y_train_labels = torch.tensor(df_train['label'].values, dtype=torch.long)
y_val_labels = torch.tensor(df_val['label'].values, dtype=torch.long)
y_test_labels = torch.tensor(df_test['label'].values, dtype=torch.long)
pre_embedding_layer = nn.Embedding(len(main_vocab), INPUT_DIM_FOR_MODEL, padding_idx=main_vocab['<PAD>'])

with torch.no_grad(): # Без этого во время обучения модель падает, тк пытается сделать backward по
    X_train_final = pre_embedding_layer(torch.tensor(X_train_ids, dtype=torch.long)).to(torch.float32)
    X_val_final = pre_embedding_layer(torch.tensor(X_val_ids, dtype=torch.long)).to(torch.float32)
    X_test_final = pre_embedding_layer(torch.tensor(X_test_ids, dtype=torch.long)).to(torch.float32)

X_train_final.shape, y_train_labels.shape, X_val_final.shape,  y_val_labels.shape, X_test_final.shape, y_test_labels.shape

/Users/gab1k/Desktop/monkey_coding_dl_project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /Users/gab1k/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Используем устройство: cpu
Размер обучающей выборки до обрезания: 25000
Размер валидационной выборки до обрезания: 12500
Размер тестовой выборки до обрезания: 12500
---------------------------------------
Итоговый размер обучающей выборки: 10000
Итоговый размер валидационной выборки: 5000
Итоговый размер тестовой выборки: 5000


(torch.Size([10000, 300, 64]),
 torch.Size([10000]),
 torch.Size([5000, 300, 64]),
 torch.Size([5000]),
 torch.Size([5000, 300, 64]),
 torch.Size([5000]))